# Proceso ETL para `users_items`

Este notebook documenta el proceso de Extracción, Transformación y Carga (ETL) aplicado al dataset `user_items`, crucial para la preparación de los datos relacionados con los artículos que los usuarios de Steam poseen, juegan y cómo interactúan con ellos. Este análisis proporciona insights valiosos sobre el comportamiento de los usuarios y sus preferencias de juego.


Librerías requeridas

In [1]:
from utils.utils import cargar_json_gz
import pandas as pd
users_items = cargar_json_gz("../data/users_items.json.gz", way=1)
users_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [2]:
users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


## Transformación de Datos

### Desanidamiento de `items` 

Desanidamos la columna 'items' y convertimos los diccionarios en columnas separadas

In [3]:
items_exploded = users_items.explode('items')
items_normalized = pd.json_normalize(
    items_exploded['items']).set_index(items_exploded.index)
items_transformed = pd.concat(
    [items_exploded.drop('items', axis=1), items_normalized], axis=1)

In [4]:
items_transformed

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330,All Is Dust,0.0,0.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490,One Way To Die: Steam Edition,3.0,3.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570,You Have 10 Seconds 2,4.0,4.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,519140,Minds Eyes,3.0,3.0


### Limpieza y Filtrado 



In [5]:
# Eliminacion de columnas no relevantes
columns_to_drop = ['items_count', 'steam_id',
                   'user_url', 'item_name', 'playtime_2weeks']
items_transformed.drop(columns=columns_to_drop, inplace=True)

# Filtrado de registros donde el tiempo de juego sea mayor a 0
items_transformed = items_transformed[items_transformed['playtime_forever'] > 0]

# Conversión de 'playtime_forever' a entero
items_transformed['playtime_forever'] = items_transformed['playtime_forever'].astype(
    'int64')

# Eliminación de duplicados y valores nulos
items_final = items_transformed.drop_duplicates(subset=['user_id', 'item_id']).dropna()

items_final

C:\Users\fcastano5\AppData\Local\Temp\ipykernel_2208\2876383580.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_transformed['playtime_forever'] = items_transformed['playtime_forever'].astype(


,user_id,item_id,playtime_forever
0,76561197970982479,10,6
0,76561197970982479,30,7
0,76561197970982479,300,4733
0,76561197970982479,240,1853
0,76561197970982479,3830,333
...,...,...,...
88308,76561198329548331,304930,677
88308,76561198329548331,227940,43
88308,76561198329548331,388490,3
88308,76561198329548331,521570,4


## Almacenamiento de Datos

In [6]:
ruta_guardado = "../data/users_items.pkl"

items_final.to_pickle(ruta_guardado)

In [7]:
# Verificación de que el archivo se haya guardado correctamente

items_final = pd.read_pickle(ruta_guardado)
items_final

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
0,76561197970982479,30,7
0,76561197970982479,300,4733
0,76561197970982479,240,1853
0,76561197970982479,3830,333
...,...,...,...
88308,76561198329548331,304930,677
88308,76561198329548331,227940,43
88308,76561198329548331,388490,3
88308,76561198329548331,521570,4



# Reporte Final del Procesamiento para `users_items`

1. **`items`**:
   - **Desanidamiento**: La columna `items`, originalmente conteniendo listas de diccionarios con información detallada sobre los juegos poseídos por cada usuario, fue desanidada. Esto implicó convertir cada elemento de la lista en una fila separada y luego expandir cada diccionario en columnas independientes, facilitando el análisis por juego.
 
2. **`playtime_forever`**:
   - **Filtrado y Conversión**: Se filtraron los registros para incluir solo aquellos juegos con tiempo de juego (`playtime_forever`) mayor a cero, indicando interacción real. Posteriormente, se convirtió esta columna a tipo entero para un análisis cuantitativo preciso.

3. **Columnas Eliminadas**:
   - **Limpieza de Dataset**: Se eliminaron varias columnas no relevantes para el análisis, como `items_count`, `steam_id`, `user_url`, `item_name`, y `playtime_2weeks`, simplificando así el DataFrame y enfocándose en los datos más impactantes para el estudio.

4. **Duplicados y Valores Nulos**:
   - **Eliminación**: Se eliminaron filas duplicadas y se manejaron los valores nulos, asegurando la integridad y la calidad del dataset para análisis futuros.

5. **`user_id` y `item_id`**:
   - **Preservación**: Estas columnas se mantuvieron intactas durante el proceso, ya que son fundamentales para identificar de manera única a los usuarios y los juegos, respectivamente.

6. **Almacenamiento de Datos**:
   - **Uso de Pickle**: El DataFrame transformado y limpio se almacenó en formato Pickle (`users_items.pkl`). Este formato fue elegido por su eficiencia en la serialización de estructuras de datos de Python, facilitando la recuperación y manipulación de los datos para análisis futuros. Se verificó la correcta creación y recuperación del archivo para asegurar la integridad de los datos almacenados.

